In [24]:
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score,precision_score,recall_score, roc_curve, classification_report,precision_recall_curve

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
d_path = '/content/drive/MyDrive/Colab Notebooks/Dacon/'
X_train = pd.read_csv(d_path + 'train_features.csv')
y_train = pd.read_csv(d_path + 'train_labels.csv')
X_test = pd.read_csv(d_path + 'test_features.csv')
submission = pd.read_csv(d_path + 'sample_submission.csv')

display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629


,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043


In [5]:
# X_train = pd.read_csv('train_features.csv')
# y_train = pd.read_csv( 'train_labels.csv')
# X_test = pd.read_csv('test_features.csv')
# submission = pd.read_csv('sample_submission.csv')

In [6]:
X_pivot_train = pd.pivot_table(data = X_train, # X_train의 데이터를 통해서
                               values = X_train.columns[2:],  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var'           # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = X_test, #
                               values = X_test.columns[2:], 
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

display(X_pivot_train.head(3))
display(X_pivot_test.head(3))

sum                          ...          var                           
         acc_x       acc_y       acc_z  ...         gy_x          gy_y         gy_z
id                                      ...                                        
0   558.797337 -131.082711 -222.252919  ...   176.470384    590.513292   638.834979
1  -459.948117 -190.354639   -2.534051  ...  6279.700472   9217.015511  5707.098884
2    23.901616  -49.441742  375.607013  ...   646.325142  14150.683677   193.775778

[3 rows x 42 columns]

sum                          ...          var                          
           acc_x       acc_y       acc_z  ...         gy_x         gy_y         gy_z
id                                        ...                                       
3125 -611.238360  -11.744605 -139.355669  ...   166.357553  1023.553453   150.102867
3126 -313.705824  367.296809  -42.655405  ...  2665.150566  2089.066820  3795.159662
3127  304.167948  542.291164  -84.658968  ...   518.511372   181.383940   530.909012

[3 rows x 42 columns]

In [7]:
X_pivot_train.columns[:5]

MultiIndex([('sum', 'acc_x'),
            ('sum', 'acc_y'),
            ('sum', 'acc_z'),
            ('sum',  'gy_x'),
            ('sum',  'gy_y')],
           )

In [8]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
display(X_pivot_train.head())

,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


In [9]:
X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

display(X_pivot_train.head())
display(X_pivot_test.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,3125,-611.238360,-11.744605,-139.355669,-1911.076959,1639.123438,-1200.410049,-1.018731,-0.019574,-0.232259,-3.185128,2.731872,-2.000683,-1.064222,-0.005735,-0.268442,-3.770150,0.108956,-1.607847,-1.564000,-0.470937,-0.573836,-50.429364,-81.607713,-35.446915,-0.275446,0.228040,0.286182,56.953059,96.185341,49.981455,0.236232,0.091641,0.174672,12.897967,31.993022,12.251648,0.055806,0.008398,0.030510,166.357553,1023.553453,150.102867
1,3126,-313.705824,367.296809,-42.655405,-10644.915365,4184.863263,-2162.747150,-0.522843,0.612161,-0.071092,-17.741526,6.974772,-3.604579,-0.677411,0.606215,-0.026089,-14.305258,-0.974696,-10.833508,-1.929033,-0.200678,-1.212052,-273.572486,-97.100707,-147.597574,0.627571,1.708743,0.671876,132.830402,241.240196,169.417650,0.539688,0.333015,0.383800,51.625096,45.706311,61.604867,0.291264,0.110899,0.147302,2665.150566,2089.066820,3795.159662
2,3127,304.167948,542.291164,-84.658968,-1307.846921,-1350.871152,-235.904841,0.506947,0.903819,-0.141098,-2.179745,-2.251452,-0.393175,0.497370,0.931239,-0.156647,0.476247,-2.319172,-3.087735,-0.792916,0.219008,-0.484614,-99.799971,-98.420987,-154.477074,2.972063,1.941820,0.644154,160.426058,74.530763,97.211730,0.219934,0.191485,0.152077,22.770845,13.467885,23.041463,0.048371,0.036666,0.023128,518.511372,181.383940,530.909012
3,3128,-346.561617,-366.333946,18.891749,485.147442,-1790.981310,-14.590798,-0.577603,-0.610557,0.031486,0.808579,-2.984969,-0.024318,-0.880541,-0.507927,-0.092860,1.457625,-0.269600,-0.404583,-1.045889,-1.294482,-0.469924,-229.072919,-168.031080,-117.297766,0.337281,-0.258476,0.702574,119.527887,118.268797,167.860762,0.431713,0.233601,0.326569,42.818157,45.069932,37.967372,0.186376,0.054569,0.106647,1833.394532,2031.298793,1441.521366
4,3129,-443.184021,109.521180,240.781103,-858.922755,865.419381,3447.298941,-0.738640,0.182535,0.401302,-1.431538,1.442366,5.745498,-0.703842,0.122280,0.432678,-3.066063,1.631638,1.866352,-2.153047,-0.860883,-0.631258,-345.447240,-223.475411,-125.598600,0.015642,1.562602,1.037876,366.167357,226.728939,138.130133,0.305797,0.314294,0.261848,92.301963,67.911174,43.353007,0.093512,0.098781,0.068565,8519.652350,4611.927587,1879.483194


In [10]:
X_pivot_train.shape

(3125, 43)

In [11]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [12]:
ftr = X_pivot_train
target = y_train['label']

In [13]:
X_train, X_val, y_train, y_val = train_test_split(ftr, target, test_size=0.2, random_state=156)

lgbm_wrapper = LGBMClassifier(n_estimators=1000, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30)

evals = [(X_val, y_val)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_pivot_test)
pred_proba = lgbm_wrapper.predict_proba(X_pivot_test)[:,:]

[1]	valid_0's multi_logloss: 2.54967	valid_0's multi_logloss: 2.54967
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.44754	valid_0's multi_logloss: 2.44754
[3]	valid_0's multi_logloss: 2.36224	valid_0's multi_logloss: 2.36224
[4]	valid_0's multi_logloss: 2.28827	valid_0's multi_logloss: 2.28827
[5]	valid_0's multi_logloss: 2.22081	valid_0's multi_logloss: 2.22081
[6]	valid_0's multi_logloss: 2.16124	valid_0's multi_logloss: 2.16124
[7]	valid_0's multi_logloss: 2.10861	valid_0's multi_logloss: 2.10861
[8]	valid_0's multi_logloss: 2.0579	valid_0's multi_logloss: 2.0579
[9]	valid_0's multi_logloss: 2.00773	valid_0's multi_logloss: 2.00773
[10]	valid_0's multi_logloss: 1.96496	valid_0's multi_logloss: 1.96496
[11]	valid_0's multi_logloss: 1.92285	valid_0's multi_logloss: 1.92285
[12]	valid_0's multi_logloss: 1.88298	valid_0's multi_logloss: 1.88298
[13]	valid_0's multi_logloss: 1.84782	valid_0's multi_logloss: 1.84782
[14]	valid_0's multi_logl

In [14]:
print(pred_proba.shape)

(782, 61)


---------------------------------------

In [16]:
def SCORES(y_val, pred, proba) :
    #acc = accuracy_score(y_val, pred)
    f1 = f1_score(y_val, pred, average='macro')
    #print(y_val[:10])
    #print(proba[:10])
    auc = roc_auc_score(y_val, proba, average='macro', multi_class='ovo')

In [20]:
print(y_val.shape)
print(pred_proba.shape)
print(preds.shape)

(625,)
(782, 61)
(782,)


In [26]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=30)

In [28]:
my_hyper_param = {
    "n_estimators"      :[300], 
    "max_depth"         :[9],
    "min_samples_leaf"  :[1,3,5],
}

In [29]:
from sklearn.metrics import make_scorer

gcv_model = GridSearchCV(rf_model, param_grid=my_hyper_param, scoring='roc_auc_ovr', refit=True, cv=5, verbose=0)
#scoring=make_scorer(recall_score, average='micro')

gcv_model.fit(X_train, y_train)

print("best_estimator:", gcv_model.best_estimator_)
print("best_params:",    gcv_model.best_params_)
print("best_score:" ,    gcv_model.best_score_)

best_estimator: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
best_params: {'max_depth': 9, 'min_samples_leaf': 3, 'n_estimators': 300}
best_score: 0.9627337060556869


In [30]:
rfc = rf_model = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=3, n_estimators=300,max_depth=9)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [31]:
lgb = LGBMClassifier(n_estimators=1000, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30)

evals = [(X_val, y_val)]
lgb.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)

[1]	valid_0's multi_logloss: 2.54967	valid_0's multi_logloss: 2.54967
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.44754	valid_0's multi_logloss: 2.44754
[3]	valid_0's multi_logloss: 2.36224	valid_0's multi_logloss: 2.36224
[4]	valid_0's multi_logloss: 2.28827	valid_0's multi_logloss: 2.28827
[5]	valid_0's multi_logloss: 2.22081	valid_0's multi_logloss: 2.22081
[6]	valid_0's multi_logloss: 2.16124	valid_0's multi_logloss: 2.16124
[7]	valid_0's multi_logloss: 2.10861	valid_0's multi_logloss: 2.10861
[8]	valid_0's multi_logloss: 2.0579	valid_0's multi_logloss: 2.0579
[9]	valid_0's multi_logloss: 2.00773	valid_0's multi_logloss: 2.00773
[10]	valid_0's multi_logloss: 1.96496	valid_0's multi_logloss: 1.96496
[11]	valid_0's multi_logloss: 1.92285	valid_0's multi_logloss: 1.92285
[12]	valid_0's multi_logloss: 1.88298	valid_0's multi_logloss: 1.88298
[13]	valid_0's multi_logloss: 1.84782	valid_0's multi_logloss: 1.84782
[14]	valid_0's multi_logl

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=30,
               min_child_samples=320, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=100,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

In [43]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('rfc', rfc), ('lgb', lgb)], voting='soft')
eclf.fit(X_train, y_train)
y_prob=eclf.predict_proba(X_pivot_test)

In [44]:
y_prob.shape

(782, 61)

In [ ]:
print(y_prob)

In [45]:
submission.iloc[:,1:] = y_prob
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000801,0.000402,0.002477,0.002556,1.338077e-08,1.755895e-08,0.000872,0.000483,0.001644,0.031724,0.331478,0.038715,2.090427e-03,0.054870,0.031027,0.000473,0.000393,1.302061e-04,7.659483e-12,1.988195e-07,4.555181e-04,0.003922,0.001036,1.500363e-03,0.139347,4.032991e-13,0.056860,0.001058,0.000462,7.571129e-04,0.016284,0.005295,0.006214,0.001742,0.003863,0.000831,0.022394,0.000356,0.002128,5.798139e-04,0.000317,0.000055,0.004686,0.002795,0.000868,4.129660e-03,3.553653e-10,0.000180,0.003707,0.004380,0.000243,0.118730,0.003444,0.002708,0.004215,0.000829,0.000607,0.001079,0.063491,2.333048e-14,0.018319
1,3126,0.002315,0.000166,0.000206,0.000857,4.793208e-03,2.575470e-04,0.000042,0.000661,0.000822,0.002074,0.001758,0.002279,7.912945e-13,0.000595,0.000258,0.000678,0.002520,9.072280e-14,1.501989e-04,3.234056e-12,8.529494e-04,0.001767,0.004579,2.839538e-03,0.001171,4.426120e-05,0.929690,0.000053,0.000169,2.020464e-14,0.002253,0.002272,0.000566,0.001818,0.002504,0.000798,0.002283,0.000005,0.000638,2.253550e-09,0.000997,0.000729,0.001169,0.001447,0.000064,1.429335e-06,1.131947e-05,0.000197,0.000131,0.005040,0.007290,0.000504,0.000004,0.000715,0.000048,0.001252,0.000022,0.003606,0.000302,1.408805e-04,0.001595
2,3127,0.003455,0.034498,0.002269,0.003164,7.252128e-10,7.497669e-04,0.426285,0.010520,0.007610,0.002817,0.002326,0.001246,4.499373e-04,0.000913,0.003244,0.000637,0.011432,2.016378e-03,2.815357e-12,1.115114e-10,2.228362e-04,0.002841,0.000138,1.716727e-04,0.004612,1.278927e-04,0.061367,0.024670,0.002294,7.927362e-03,0.004012,0.001375,0.008104,0.006094,0.006000,0.003841,0.013366,0.013151,0.002755,9.761214e-04,0.012644,0.002002,0.006389,0.013611,0.009723,5.984262e-02,1.214986e-03,0.092152,0.021468,0.015586,0.000943,0.001867,0.000831,0.000429,0.033501,0.003317,0.014580,0.000247,0.002663,1.628150e-02,0.013031
3,3128,0.004404,0.003597,0.003235,0.001161,2.122822e-03,1.087503e-03,0.000090,0.004133,0.000776,0.001413,0.001062,0.001279,6.535950e-05,0.000364,0.000142,0.000633,0.006881,1.071233e-04,9.005357e-05,1.023579e-09,6.473577e-04,0.007353,0.003035,2.341132e-03,0.001036,2.723933e-05,0.888144,0.000689,0.000364,2.366309e-04,0.002865,0.002394,0.001473,0.004537,0.003692,0.000711,0.002715,0.000374,0.000447,4.094358e-05,0.002200,0.002092,0.003766,0.007659,0.000450,4.103654e-04,8.089867e-06,0.000069,0.000997,0.005584,0.007935,0.000173,0.000046,0.000109,0.000661,0.001411,0.000234,0.004551,0.000226,2.240346e-05,0.005633
4,3129,0.007380,0.000184,0.000196,0.000713,4.644520e-03,2.424700e-04,0.000035,0.000560,0.001415,0.001258,0.001620,0.001291,1.929358e-13,0.000293,0.000195,0.002945,0.001674,1.924563e-06,5.254800e-04,3.797139e-11,7.371402e-04,0.001394,0.005470,2.341614e-03,0.000995,7.154150e-05,0.923698,0.000018,0.000122,2.898178e-15,0.001470,0.001531,0.000581,0.001708,0.003670,0.000441,0.001708,0.000003,0.000226,2.632363e-07,0.001071,0.000641,0.001129,0.001324,0.000064,3.222707e-11,8.147884e-06,0.000102,0.000092,0.004755,0.012520,0.000039,0.000002,0.000109,0.000044,0.000241,0.000021,0.004602,0.000244,1.307648e-04,0.001507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.003324,0.000253,0.000134,0.000906,3.756312e-02,4.922705e-04,0.000037,0.000515,0.001758,0.002831,0.003012,0.002243,8.113650e-12,0.000192,0.001565,0.003734,0.001515,1.924558e-06,9.031620e-04,1.499935e-09,9.762454e-04,0.001062,0.012009,1.950337e-03,0.000719,1.651723e-03,0.866751,0.000005,0.000077,2.898427e-12,0.003708,0.002707,0.003845,0.001533,0.007107,0.000574,0.001296,0.000001,0.000740,7.633479e-10,0.000933,0.001236,0.000840,0.000962,0.000245,2.826879e-09,3.285804e-06,0.000061,0.000060,0.004869,

In [48]:
submission.to_csv(d_path + 'baseline_rf11.csv', index=False)